<a href="https://colab.research.google.com/github/sdrangan/wirelesscomm-soln/blob/master/unit01_antennas/lab_uav_ant_soln_sn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating a 28 GHz antenna for a UAV

For complex antennas, it is often necessary to perform detailed EM simulations in third-party software such as HFSS and then import the results into [Nvidia sionna](https://nvlabs.github.io/sionna/) for analysis. In this lab, we will import HFSS simulation data for a 28 GHz antenna designed for a UAV (unmanned aerial vehicle or drone). Antenna modeling is particularly important for mmWave aerial links since the directivity gain is necessary to overcome the high isotropic path loss of mmWave frequencies. Also, UAVs can be an arbitrary orientation and it is important to model the cases when the UAV is out of the beamwidth of the antenna.

In going through this lab, you will learn to:
* Import data from an EM simulation data
* Compute directivity from the E-field values
* Create custom antennas that can be used in Nvidia sionna
* Display 2D and 3D antenna patterns
* Compute the half-power beamwidth (HPBW) of an antenna
* Compute fractions of power in angular areas
* Estimate the path loss along a path


The data for this lab was generously donated by Vasilii Semkin of VTT and taken from the paper:

> Xia, W., Polese, M., Mezzavilla, M., Loianno, G., Rangan, S., & Zorzi, M.  [Millimeter wave remote UAV control and communications for public safety scenarios](https://ieeexplore.ieee.org/abstract/document/8824919), in IEEE International Conference on Sensing, Communication, and Networking (SECON), 2020

The paper performs EM simulations on a ciccularly polarized 28 GHz antenna mounted to the bottom of a commercial DJI Matrice 100 quadrocopter. An image of the drone with the antenna and its pattern is shown in the following picture.

<img src="https://github.com/sdrangan/wirelesscomm/blob/master/unit01_antennas/CP_patch_downwards_m100_3D_coord.png?raw=true" alt="Alt Text" width="500" height="300">


*Submission*: Complete all the sections marked `TODO`, and run the cells to make sure your scipt is working. When you are satisfied with the results,  publish your code to generate an html file. Print the html file to PDF and submit the PDF.


## Importing the Sionna Package
We first follow the demo and import the sionna package.  If you are running this notebook on your own machine, you should install sionna.  The code below will install sionna on Google colab.

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    !pip install sionna
    #os.system("pip install sionna")
    import sionna as sn

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

While Nvidia's sionna has enormous number of features, it does not have a way to directly create an antenna with a custom antenna pattern.  So, I have created a class to help with custom antenans in `sionnautils` package. We import this package as well.

In [ ]:
# Import Sionna utils from the wireless class
try:
    import sionnautils
except ImportError as e:
    # Install Sionna if package is not already installed
    !pip install git+https://github.com/sdrangan/wirelesscomm.git
    import sionnautils


# Load the data

To begin analyzing the UAV antenna pattern, we first download the data file with the following command.

In [ ]:
import os
import requests

# Updated file_url to point to the raw file on GitHub
file_url = "https://github.com/sdrangan/wirelesscomm/blob/master/unit01_antennas/uav_patch_bottom.csv"
file_url = file_url+"?raw=true"
file_name = "uav_patch_bottom.csv"

if not os.path.exists(file_name):
  try:
    response = requests.get(file_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(file_name, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"File '{file_name}' downloaded successfully.")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")

else:
  print(f"File '{file_name}' already exists.")

Read the csv file as a pandas data frame, `df`, and display the first few rows with the `df.head()` command.

In [ ]:
import pandas as pd

# TODO
#   df = pd.read_csv(...)
#   df.head()

You should see that the dataframe has columns for the azimuth and elevation angles as well as the real and imaginary components of the E-field for both the H and V polarizations.  Load these into numpy arrays.  Then, compute the complex E-fields, `Ev` and `Eh`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# TODO:  Read the columns from the dataframe
# into numpy arrays
#   az = ...
#   el = ...
#   Ev_re = ...
#   Ev_im = ...
#   Eh_re = ...
#   Eh_im = ...

# TODO:  Compute the complex E-fields
#   Ev = ...
#   Eh = ...


## Computing the Directivity

Recall that the directivity is proportional to the E-field.  That is,
~~~
   dir = c*Epow
~~~
where `c` is some constant and:
~~~
  Epow = abs(Ev)**2 + abs(Eh)**2
~~~
To compute the constant `c`, we know that the average of the directivity `dir * cos(el)` over a spherical integral must be one. Hence, if we take mean over the discrete values, you should have that:
~~~
    mean(dir.*cos(el)) / mean(cos(el)) = 1
~~~
You can use this relation to find the scale factor, `c`.

TODO: Find the scale factor `c` and compute a vector, `dir`, with the directivity of each point in dBi. Remember, numpy's `cos()` function takes the argument in radians.


In [ ]:
# TODO:  Find and print c
#   Epow = ...
#   c = ...
#   print('c=...)


# TODO:  Compute the directivity in dBi
#   dir = ...

# TODO:  Print the max directitivy

We will next plot the directivity.  In the data, you will see there are:
* 181 azimuth angular steps from -180 to 180 in 2 degrees steps
* 91 elevation angle steps from -90 to 90, also in 2 degree steps

Rearrange the directivity to a `91 x 181` matrix so that the azimuth angles varies over the columns and the elevation angles vary over the rows.  Put the -90 degree at the top.  In this case, the -90 degree elevation is actually 0 degree inclination since, and 90 degree elevation is 180 degree inclination (pointing in the negative z-axis).  The reason is that the measurements were done upside down.

In [ ]:
naz = 181
nel = 91

# TODO:  Reshape the directivity to a grid
#  dir_grid = np.reshape(dir, ...)

Now use the `plt.imshow` method to plot a heat map of the directivity.  Set the limits of the axes with the `extent` parameter and label the axes.  Add a colorbar.  You should see that the directivity is strong in the lower hemisphere, since the antenna is designed to point down for ground coverage.

In [ ]:
# TODO:
#   plt.imshow(...)


## Creating a Custom Antenna in Sionna

To create a custom antenna from a grid of points, I have created a class `PatternInterpGrid` which is a tensorflow interpolation on a grid.  You can see the code if you like in the [github code for antennas](https://github.com/sdrangan/wirelesscomm/blob/master/sionnautils/antenna.py).  In this lab, you do not need to modify this code -- you can just use it.

To use it we first need to scale the `Ev` and `Eh` arrays by the constant `sqrt(c)` so that `Epow` has an average value of one.  Then, reshape them to `(nel,naz)` as you did for the directivity.
  

In [ ]:
# TODO:  Scale and reshape Ev and Eh
#   Ev_grid = ...
#   Eh_grid = ...


# Convert to tensorflow constants
Ev_grid = tf.constant(Ev_grid, dtype=tf.complex64)
Eh_grid = tf.constant(Eh_grid, dtype=tf.complex64)

We can now create a custom antenna as follows:

In [ ]:
import sionnautils
from sionnautils.antenna import PatternInterpGrid

pg = PatternInterpGrid(Ev_grid, Eh_grid, dtype_real=tf.float32)

We can plot the antenna pattern with the sionna visualization function.  You will see that the pattern is a bit "spiky" since the interpolation is nearest neighbor.  I may improve the interpolation to find a smoother algorithm.  In any case, you should see that the antenna pattern gives good coverage in the lower hemisphere.

In [ ]:
sn.rt.antenna.visualize(pg.pattern)

## Plotting the Directivity on a Flight Path
We will follow the demo in class to plot the gain along a flight path.

We first create an empty scene.

In [ ]:
import sionnautils.custom_scene as custom_scene
scene = sn.rt.load_scene(custom_scene.get_scene('empty_scene'))

# Set the scene frequency
fc = 3.5e9
scene.frequency = fc

Next set the TX position at the origin.  Set the RX positions corresponding on `npts` on a line of flight from `rxstart` to `rxend`.  Plot the flight trajectory.

In [ ]:
# TODO:  Set the transmitter position at [0,0,0]
#  txpos = ...


# RX path parameters
npts = 100;
rxstart = np.array([-300, -20, 30])
rxend = np.array([300, 20, 100])

# TODO:  Set the RX positions where rxpos[i,:] is the position at point i
#   rxpos = ...
]

# TODO:  Plot the fight trajectory
# Creating the 3D plot


Next set the TX array to a "tr38901" pattern with vertical polarization.  This is a good model for a commercial base station.  Set the RX array for the UAV to the custom pattern.  Note that for a custom antenna, you do not need to set the polarization parameter.

In [ ]:
# TODO:  Configure antenna array for all transmitters
#   scene.tx_array = sn.rt.PlanarArray(
#                        pattern='tr38901', polarization='V', ...)


# TODO:  Configure antenna array for all the receivers using the custom pattern
#   scene.rx_array = sn.rt.PlanarArray(
#                        pattern=pg.pattern, ...)


Next, we add the transmitter (the ground base station) and receivers (the UAV flight points).  For the ground base station we will orient it so that it is pointed upwards by having it look at a point directly above it.

In [ ]:
# TODO:  Create transmitter and add it to the scene
#   tx = sn.rt.Transmitter(...)
#   scene.add(...)


# TODO:  Point the TX to look at a point directly above it, say [0,0,1]

# TODO:  Add the receivers

Compute the propagation paths from the TX to the receivers.  Set the `paths.normalize_delays=False` and use the `paths.cir()` method to get the gain and delays of the paths.  Convert the gains and delays to numpy vectors.

In [ ]:
# TODO:  Compute the paths
#   paths = scene.compute_paths()
#   a, tau = ...


# TODO:  Convert to numpy vectors
#   a = ...
#   tau = ...

Plot the delay in micro-seconds vs. the x-position of the UAV.  Compare to the expected delay.  Label the axes.

In [ ]:
# TODO:
#   tau_est = ...
#   plot(...)


Plot the gain along the path vs. the x-position of the UAV.  Also plot the theoretical omni-directional gain.  You should see that the measured gain is higher than the omni-directional gain when the UAV is directly above the base station since both antennas are pointing at one another.  However, as it moves away, the directivity of both antennas drop and the gain decreases.

In [ ]:
# TODO:  Compute the gain in dB from the `a` vector
#  gain = ...

# TODO:  Compute the theoretical free space path loss with omni antenna
#   gain_omni = ...


# TODO:  Plot the gains vs. UAV x-position


## Measuring the Half-Power Beamwidth

Finally, we will measure the half-power beamwidth.  Since the antenna is mostly directed in the elevation direction, we will measure the pattern in the elevation.  To do this:
*  Measure the electric fields, `Ev` and `Eh`, for `theta` between 0 and pi a|nd `phi=0`.

In [ ]:
ntheta = 180

# TODO:
#  theta = tf.linspace(...)
#  phi = tf.zeros(...)

# TODO:  Get the Ev and Eh from the pg.pattern function
#  Ev, Eh = ...
#  convert to numpy arrays


# TODO:  Compute the directivity
#  D = ...


# TODO:  Plot the directivity vs theta



Since the gain has a maximum at `theta=180` degrees, the 3dB HPBW is the defined as
~~~
    hpbw = 2*(180-theta3db)
~~~
where `theta3db` is the angle at which the directivity is 3dB below the maximum.  Find `theta3db` and `hpbw`.

In [ ]:
# TODO:
#  theta3db = ...
#  hpbw = ...

# Find the value i where D[i] >= Dmax - 3

